In [1]:
import pandas as pd
import numpy as np
import sys, os , time
from tabulate import tabulate

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(folder+file_name)
    print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
methods_list = ["Ann(tr)","softMV","hardMV", "DS", "Raykar", "OursGlobal", ""]
methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","",]

def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(folder+archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return np.mean(result_list)
    except:
        return "-"
!ls

final_CIFAR	   LabelMe_v2		 old_exec	     run_simple.sh
LabelMe		   LabelMe_v2_noflatten  Plot_Results.ipynb  simple
LabelMe_aux	   LabelMe_v3		 run_CIFAR.sh	     simple_norm
LabelMe_noflatten  LabelMe_v3_noflatten  run_LabelMe.sh


In [3]:
!ls LabelMe_v3_noflatten
folder = "LabelMe_aux/"
files = sorted(os.listdir(folder))

LabelMe_DS_test.csv		 LabelMe_Raykar_test.csv
LabelMe_DS_train.csv		 LabelMe_Raykar_trainAnn.csv
LabelMe_hardMV_test.csv		 LabelMe_Raykar_train.csv
LabelMe_hardMV_train.csv	 LabelMe_softMV_test.csv
LabelMe_OursGlobal_testAux.csv	 LabelMe_softMV_train.csv
LabelMe_OursGlobal_test.csv	 LabelMe_UpperBound_test.csv
LabelMe_OursGlobal_trainAnn.csv  LabelMe_UpperBound_train.csv
LabelMe_OursGlobal_train.csv	 result.txt


In [10]:
for s in range(1,10):
    print("\n****************************** Scenario "+str(s)+" ******************************")
    scenario_files = find_match("s"+str(s)+".csv",files)
    for method in methods_list[1:-1]:
        aux = find_match(scenario_files,files)
        for value in aux:
            plot_files(value)
        print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="soft-MV"))




****************************** Scenario 1 ******************************

File to show:  simCIFAR_softMV_test_s1.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Global           0.50558       0.50558
====  ============  ==========  ============

File to show:  simCIFAR_softMV_train_s1.csv
====  ============  ==========  ============  ============  =========  =============  ==========
  ..  Unnamed: 0      Accuracy    F1 (micro)    Mean NormF    Mean JS    Wmean NormF    Wmean JS
====  ============  ==========  ============  ============  =========  =============  ==========
   0  Global          0.513692      0.513692      0.147445   0.234836       0.147411     0.23472
====  ============  ==========  ============  ============  =========  =============  ==========

File to show:  simCIFAR_hardMV_test_s1.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (

In [14]:
def compare_table(tipo,repre="normal"):
    t = pd.DataFrame()
    t["Methods"] = methods_list
    for s in range(1,10):
        t["Scenario "+str(s)] = ["-" for _ in range(len(methods_list))]
        if tipo == 'acc':
            t.loc[0,"Scenario "+str(s)] =  calculate_aux("result_scenario"+str(s)+".txt",what_to_do="mv")

        scenario_files = find_match("s"+str(s)+".csv",files)

        for k, method in enumerate(methods_list[1:]):
            aux = find_match(method,scenario_files)
            if tipo == "acc": #como normal?
                test_aux = find_match("_test_",aux)
                if len(test_aux) != 0:
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+test_aux[0])["Accuracy"][0], 4)
                    
            elif tipo == 'norm' or tipo=='js':
                kl_aux = find_match("_train_",aux)
                if len(kl_aux) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    if tipo == 'norm':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"NormF"][0] ,4)
                        except:
                            pass
                    elif tipo=='js':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"JS"][0] ,4)
                        except:
                            pass
            elif tipo == 'accann':
                train_ann = find_match("_trainAnn_",aux)
                if len(train_ann) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_ann[0])[to_extract+"ACC imiting Annot"][0] ,4)
            elif tipo == 'rmse':
                train_aux = find_match("_train_",aux)
                if len(train_aux) != 0:
                    try:
                        t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_aux[0])["Average RMSE"][0] ,4)
                    except:
                        pass
            elif tipo == "epochs":
                t.loc[k+1,"Scenario "+str(s)]=calculate_aux("result_scenario"+str(s)+".txt",what_to_do="epochs",model=methods_list_aux[k])
    return t
t_acc = compare_table("acc","normal")
t_norm_mean = compare_table("norm","mean")
t_js_mean = compare_table("js","mean")
t_norm_wmean = compare_table("norm","wmean")
t_js_wmean = compare_table("js","wmean")
t_acc

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,,-,-,-,-,-,-,-,-,-


In [12]:
t_js_mean #normal

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.2348,-,-,0.2307,0.212,-,0.2144,-,-
2,hardMV,0.1938,-,-,0.1984,0.2369,-,0.1448,-,-
3,DS,0.0044,-,-,0.0948,-,-,0.1087,-,-
4,Raykar,0.0042,-,-,0.0569,-,-,0.0475,-,-
5,OursGlobal,0.1966,-,-,0.2339,0.208,-,0.125,-,-
6,,0.0044,-,-,0.0948,0.208,-,0.1087,-,-


In [17]:
t_norm_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.1474,-,-,0.145,0.1375,-,0.1398,-,-
2,hardMV,0.1376,-,-,0.1406,0.1566,-,0.1225,-,-
3,DS,0.014,-,-,0.0638,-,-,0.0674,-,-
4,Raykar,0.013,-,-,0.0583,-,-,0.0559,-,-
5,OursGlobal,0.1401,-,-,0.1548,0.1383,-,0.1109,-,-
6,,0.014,-,-,0.0638,0.1383,-,0.0674,-,-


In [20]:
t_accAnn_mean = compare_table("accann","mean")
t_accAnn_wmean = compare_table("accann","wmean")
t_rmseAnn = compare_table("rmse","normal")
t_accAnn_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,0.6856,-,-,0.7567,0.8267,-,0.7005,-,-
5,OursGlobal,0.6392,-,-,0.6807,0.741,-,0.6494,-,-
6,,0.6392,-,-,0.6807,0.741,-,0.6494,-,-


In [35]:
t_epochs = compare_table("epochs")
t_epochs

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,,-,-,-,-,-,-,-,-,-


In [25]:
print(tabulate(t_acc,headers='keys',tablefmt='latex'))

\begin{tabular}{rlrrrrrrrll}
\hline
    & Methods    &   Scenario 1 &   Scenario 2 &   Scenario 3 &   Scenario 4 &   Scenario 5 &   Scenario 6 &   Scenario 7 & Scenario 8   & Scenario 9   \\
\hline
  0 & Ann(tr)    &     0.777371 &      0.61335 &      0.78022 &     0.771673 &     0.807082 &     0.622711 &     0.782255 & -            & -            \\
  1 & softMV     &     0.6644   &      0.6081  &      0.6572  &     0.6667   &     0.6764   &     0.5914   &     0.665    & -            & -            \\
  2 & hardMV     &     0.7248   &      0.5947  &      0.7527  &     0.7345   &     0.7297   &     0.5784   &     0.7913   & -            & -            \\
  3 & DS         &     0.9482   &      0.9429  &      0.9464  &     0.9344   &     0.9417   &     0.8413   &     0.9254   & -            & -            \\
  4 & Raykar     &     0.9415   &      0.9418  &      0.9417  &     0.9372   &     0.9308   &     0.8037   &     0.9179   & -            & -            \\
  5 & OursGlobal &     0.90

In [62]:
def compare_table(tipo,repre="normal"):
    t = pd.DataFrame()
    t["Methods"] = methods_list
    t["Train"] = ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        t.loc[0,"Train"] =  calculate_aux("result.txt",what_to_do="mv")

    for k, method in enumerate(methods_list[1:]):
        aux = find_match(method,files)
        if tipo == "acc": #como normal?
            test_aux = find_match("_test.csv",aux)
            t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[0])["Accuracy"][0], 4)
            train_aux = find_match("_train.csv",aux)
            t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[0])["Accuracy"][0], 4)

        elif tipo == 'norm' or tipo=='js':
            kl_aux = find_match("_train.csv",aux)
            if len(kl_aux) != 0:
                if repre=="wmean":
                    to_extract = "Wmean "
                elif repre=='mean':
                    to_extract="Mean "
                if tipo == 'norm':
                    try:
                        t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"NormF"][0] ,4)
                    except:
                        pass
                elif tipo=='js':
                    try:
                        t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"JS"][0] ,4)
                    except:
                        pass
        elif tipo == "epochs":
            t.loc[k+1,"Train"]=calculate_aux("result.txt",what_to_do="epochs",model=methods_list_aux[k])
    return t
t_acc = compare_table("acc","normal")
t_norm_mean = compare_table("norm","mean")
t_js_mean = compare_table("js","mean")
t_epochs = compare_table("epochs")

t_acc

,Methods,Train,Test
0,Ann(tr),0.769,-
1,softMV,0.7737,0.7767
2,hardMV,0.7589,0.7734
3,DS,0.8004,0.8109
4,Raykar,0.7887,0.7961
5,OursGlobal,0.8072,0.8181
6,,0.8004,0.8109


In [64]:
t_js_mean

,Methods,Train,Test
0,Ann(tr),-,-
1,softMV,0.2485,-
2,hardMV,0.2464,-
3,DS,0.153,-
4,Raykar,0.3112,-
5,OursGlobal,0.2156,-
6,,0.153,-


In [49]:
t_epochs

,Methods,Train,Test
0,Ann(tr),-,-
1,softMV,4.9,-
2,hardMV,5.86667,-
3,DS,5.2,-
4,Raykar,2.06167,-
5,OursGlobal,18.965,-
6,,7.3894,-


In [4]:

for method,aux_meth in zip(methods_list[1:-1],methods_list_aux[:-1]):
    aux = find_match(method,files)
    for value in aux:
        plot_files(value)
    print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model=aux_meth))


Epochs: nan
Epochs: nan
Epochs: nan
Epochs: nan

File to show:  LabelMe_OursGlobal_test.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Global          0.836672      0.836672
====  ============  ==========  ============

File to show:  LabelMe_OursGlobal_testAux.csv
====  ========  ========  =======  =========  ===========  =========  =============  =========  ==========  ============
  ..    Groups      Prob     T(g)    Entropy    Diag Mean    KL to I    I sim %(JS)    Spammer    Accuracy    F1 (micro)
====  ========  ========  =======  =========  ===========  =========  =============  =========  ==========  ============
   0         0  0.684573  39.9667   0.380325     0.922977  0.0809819       0.942426   0.911974     0.83844       0.83844
   1         1  0.315427  18.0333   0.772325     0.784268  0.257236        0.82458    0.75345      0.83367       0.83367
====  ========  ========  =====

/users/fmena/anaconda3/envs/tesis/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/fmena/anaconda3/envs/tesis/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
